In [42]:
import os
from dotenv import load_dotenv
from binance.client import Client
import pandas as pd
import datetime
import time
from pathlib import Path

In [57]:
def Binance(symbol, years):
    """
    Gets historical crypto data from the binance API
    Inputs:
        Symbol as shown on exchanges, years(int) of historical data to get the
    Outputs:
        A dataframe with date as index, close and volume data
    """

        
    years = int(years)

    # Format symbol for api call
    symbol = symbol.upper()
    binance_symbol = f"{symbol}USDT"

    # Load environment vairables
    # Set variables to binance api key env files
    load_dotenv()
    binance_api = os.getenv("BINANCE_API")
    binance_secret = os.getenv("BINANCE_SECRET")

    # Create api client variable
    client = Client(binance_api, binance_secret)

    # Create and format date
    start_date = datetime.date.today() - pd.Timedelta(weeks=52*years)
    start_date = start_date.strftime("%d %b, %Y")

    # make api call and get returned data
    candles = client.get_historical_klines(binance_symbol, Client.KLINE_INTERVAL_1DAY, limit=1000, start_str=start_date) 

    # Create and format dataframe 
    # Returns close and volume with date as index
    columns = ['Date', 'open', 'high', 'low', 'close', 'volume', 'close time', 'quote asset volume', 
    'number of trades', 'taker buy base asset volume', 'taker buy quote asset volume', 'ignore']
    df = pd.DataFrame(candles, columns=columns)

    mills = df['Date']
    date = pd.Series([datetime.datetime.fromtimestamp(mill/1000) for mill in mills])

    # Combine dataframes and drop columns
    df = pd.concat([date, df], axis=1, join='inner')
    
    # Set data type to float
    df['Date'] = df[0]
    
        # Set date index
    df = df.set_index(['Date'])
    df.index = pd.to_datetime(df.index, format='%m%Y%d')
    df.index = df.index.date
    df = df.reset_index()
    df['Date'] = df['index']
    df.drop(['index'], axis=1, inplace=True)
    
    df['Open'] = df['open'].astype(float)
    df['Low'] = df['low'].astype(float)
    df['High'] = df['high'].astype(float)
    df['Close'] = df['close'].astype(float)
    df['Volume'] = df['volume'].astype(float)
    
    df = df.drop(columns=[0, 'close', 'volume', 'open', 'high', 'low','close time', 'quote asset volume', 'number of trades', 
    'taker buy base asset volume', 'taker buy quote asset volume', 'ignore'], axis=1)


    # Rename columns with ticker values
#     df = df.rename(columns={0:'date', 'close':f'{binance_symbol}_Close', 'volume':f'{binance_symbol}_Volume'})



    return df

In [58]:
BTC_5_YEARS = Binance("BTC", 5)

In [60]:
BTC_5_YEARS.to_csv(Path('./Resources/BTC-5YRS-D.csv'))

In [59]:
BTC_5_YEARS.head()

,Date,Open,Low,High,Close,Volume
0,2017-08-16,4261.48,4200.74,4485.39,4285.08,795.150377
1,2017-08-17,4285.08,3938.77,4371.52,4108.37,1199.888264
2,2017-08-18,4108.37,3850.00,4184.69,4139.98,381.309763
3,2017-08-19,4120.98,4032.62,4211.08,4086.29,467.083022
4,2017-08-20,4069.13,3911.79,4119.62,4016.00,691.743060
